In [1]:
# set working directory :
import os

from dotenv import load_dotenv
load_dotenv()

pwd = os.getcwd() + "/../"
os.chdir(pwd)
print(os.getcwd())

/Users/carmelomileto/Documents/Projects/TickApp


In [2]:
from tickapp.clients.claude_client import ClaudeClient
from tickapp.clients.prompt_client import PromptClient

In [3]:
# 1. Init
client = ClaudeClient(api_key=os.getenv("ANTHROPIC_API_KEY"))

# 2. Login (optionnel, pour tester la connexion)
if client.login():
    print("✅ Connecté")

✅ Connecté


In [4]:
path_pictures = ["/Users/carmelomileto/.local/share/signal-cli/attachments/2ltlwohfVpAMoERvUG0j.jpg"]

In [5]:
# Générer le prompt dynamiquement avec les catégories de la base de données
prompt_client = PromptClient(
    host="localhost",
    port=5434,
    database="receipt_processing",
    user="receipt_user",
    password="SuperSecretPassword123!"
)

prompt = prompt_client.generate_prompt()
print("📝 Prompt généré avec les catégories de la base de données")
prompt

📝 Prompt généré avec les catégories de la base de données


'Analyse ce ticket de caisse et extrais toutes les informations des articles achetés.\n\nINSTRUCTIONS :\n1. Identifie le nom du magasin (Migros, Coop, Manor, Interdiscount, Landi, etc.)\n2. Identifie tous les articles listés sur le ticket (même si répartis sur plusieurs images)\n3. Pour chaque article, extrais :\n   - Le nom exact de l\'article tel qu\'écrit\n   - Le prix unitaire\n   - La quantité (si indiquée)\n   - Le prix total\n\n4. Comprends et catégorise chaque article dans UNE de ces catégories (basées sur les standards bancaires) :\n   \n   Alimentation et supermarchés:\n      - Condiments et sauces\n      - Conserves et produits secs\n      - Fruits et légumes frais\n      - Pain et viennoiseries\n      - Poisson et fruits de mer\n      - Produits laitiers et œufs\n      - Produits surgelés\n      - Pâtes, riz et céréales\n      - Snacks et confiserie\n      - Viande et charcuterie\n\n   Animalerie:\n      - Accessoires pour animaux\n      - Nourriture pour animaux\n      - S

In [6]:
# 3. Add prompt
client.add_prompt(prompt)

# 4. Add pictures
client.add_images(path_pictures)

# 5. Call Claude (ne pas reset pour pouvoir réutiliser avec call_json)
response = client.call(reset_after=False)
print(response)

{
  "magasin": {
    "nom": "Aeschbach",
    "adresse": "Avenue Louis-Casaï 27",
    "code_postal": "1211",
    "ville": "Genève",
    "pays": "CH",
    "telephone": "022 979 09 00"
  },
  "transaction": {
    "category_id": 2,
    "numero_ticket": "1755500",
    "date": "2023-11-21",
    "heure": "16:39:00",
    "mode_paiement": "carte"
  },
  "devise": "CHF",
  "articles": [
    {
      "nom": "Textile",
      "reference": "99950074-XL",
      "marque": "",
      "quantite": 1,
      "prix_unitaire": 97.30,
      "prix_total": 97.30,
      "categorie": "Vêtements et chaussures",
      "sous_categorie": "Vêtements femme",
      "tva": ""
    },
    {
      "nom": "Textile",
      "reference": "99771399M-M",
      "marque": "",
      "quantite": 1,
      "prix_unitaire": 143.20,
      "prix_total": 143.20,
      "categorie": "Vêtements et chaussures",
      "sous_categorie": "Vêtements femme",
      "tva": ""
    }
  ],
  "total": 240.50
}


In [7]:
# OU si tu veux du JSON directement (sans reset pour pouvoir réutiliser)
# Note: Si le contenu a été reset, réajouter le prompt et les images d'abord
if not client.content:
    client.add_prompt(prompt)
    client.add_images(path_pictures)

json_response = client.call_json(reset_after=True)
print(json_response)

{'magasin': {'nom': 'Aeschbach', 'adresse': 'Avenue Louis-Casaï 27', 'code_postal': '1211', 'ville': 'Genève', 'pays': 'CH', 'telephone': '022 979 09 60'}, 'transaction': {'category_id': 2, 'numero_ticket': '1755550', 'date': '2023-11-21', 'heure': '18:39:00', 'mode_paiement': 'VISA'}, 'devise': 'CHF', 'articles': [{'nom': 'Textile', 'reference': '99659071A- XL', 'marque': 'Aeschbach', 'quantite': 1, 'prix_unitaire': 97.3, 'prix_total': 97.3, 'categorie': 'Vêtements et chaussures', 'sous_categorie': 'Vêtements femme', 'tva': '8.1%'}, {'nom': 'Textile', 'reference': '92771939M- M', 'marque': 'Aeschbach', 'quantite': 1, 'prix_unitaire': 143.2, 'prix_total': 143.2, 'categorie': 'Vêtements et chaussures', 'sous_categorie': 'Vêtements femme', 'tva': '8.1%'}], 'total': 240.5}
